In [ ]:
%matplotlib inline
import sys, os, distutils.core

import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor, default_argument_parser, default_setup, launch
from detectron2.engine.hooks import HookBase
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import detection_utils as utils
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import BoxMode
from detectron2.evaluation import COCOEvaluator

import detectron2.data.transforms as T
import detectron2.utils.comm as comm

import numpy as np
import os, json, cv2, random
from matplotlib import pyplot as plt

import copy
from glob import glob
from pathlib import Path
import time
import datetime
import logging

import utility
CACHE_DIR = Path('./caches')
utility.create_directory(CACHE_DIR)
OUTPUT_DIR = Path('./output')
utility.create_directory(OUTPUT_DIR)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [ ]:
print("torch: ", torch.__version__)
print("detectron2:", detectron2.__version__)

In [ ]:
def cv2_imshow(im, title='figure', figsize=(20, 20), bgr2rgb=False):
    if isinstance(im, torch.Tensor):
        im = im.permute(1, 2, 0).cpu().numpy().astype(np.uint8)
    plt.figure(figsize=figsize)
    if bgr2rgb:
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.imshow(im)
    plt.title(title)

def get_data_dicts(coco_json_dir, img_dir):
    with open(coco_json_dir) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []

    for annotation_obj in imgs_anns["annotations"]:
        annotation_obj['bbox_mode'] = BoxMode.XYWH_ABS

    for image_obj in imgs_anns["images"]:
        record = {}
        record["file_name"] = str(Path(img_dir) / Path(image_obj['file_name']))
        record["height"] = image_obj['height']
        record["width"] = image_obj['width']
        record["image_id"] = image_obj['id']
        record["annotations"] = []
        for annotation_obj in imgs_anns["annotations"]:
            if annotation_obj['image_id'] == image_obj['id']:
                record["annotations"].append(annotation_obj)
        dataset_dicts.append(record)
    return dataset_dicts

class CustomMapper():
    def __init__(self, cfg, transform_list=[]):
        super().__init__()
        self.cfg = cfg
        self.transform_list = transform_list
        self.transform_list.append(T.ResizeShortestEdge([cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST))

    def __call__(self, dataset_dict):
        dataset_dict = copy.deepcopy(dataset_dict)
        image = utils.read_image(dataset_dict["file_name"], format="BGR")
        image, transforms = T.apply_transform_gens(self.transform_list, image)
        dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))
        annos = [
            utils.transform_instance_annotations(annotation, transforms, image.shape[:2])
            for annotation in dataset_dict.pop("annotations")
            if annotation.get("iscrowd", 0) == 0
        ]
        instances = utils.annotations_to_instances(annos, image.shape[:2])
        dataset_dict["instances"] = utils.filter_empty_instances(instances)
        return dataset_dict

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader

    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)

        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses

    def _get_loss(self, data):
        # How loss is calculated on train_loop
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced


    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

In [ ]:
n_train_samples = 4
DatasetCatalog.register(
    "mutaSCAN_train",
    lambda: get_data_dicts(f'datasets/train/coco_{n_train_samples}_samples.json', f'caches/train_samples'))
MetadataCatalog.get("mutaSCAN_train").set(thing_classes=["0", "1", "2"])

DatasetCatalog.register(
    "mutaSCAN_valid",
    lambda: get_data_dicts(f'datasets/valid/coco.json', f'caches/valid_samples'))
MetadataCatalog.get("mutaSCAN_valid").set(thing_classes=["0", "1", "2"])

dataset_dicts = get_data_dicts(f'datasets/train/coco_{n_train_samples}_samples.json', 'caches')
mutaSCAN_metadata = MetadataCatalog.get("mutaSCAN_train")

Random Visualization

In [ ]:
# train dataset
dataset_dicts = DatasetCatalog.get('mutaSCAN_train')
d = random.sample(dataset_dicts, 1)[0]
im = cv2.imread(d["file_name"])
visualizer = Visualizer(im, metadata=mutaSCAN_metadata, scale=1.0)
vis = visualizer.draw_dataset_dict(d)
cv2_imshow(vis.get_image(), bgr2rgb=True)

In [ ]:
# valid dataset
dataset_dicts = DatasetCatalog.get('mutaSCAN_valid')
d = random.sample(dataset_dicts, 1)[0]
im = cv2.imread(d["file_name"])
visualizer = Visualizer(im, metadata=mutaSCAN_metadata, scale=1.0)
vis = visualizer.draw_dataset_dict(d)
cv2_imshow(vis.get_image(), bgr2rgb=True)

Configuration for Training

In [ ]:
cfg = get_cfg()

# get configuration from model_zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")

cfg.DATALOADER.NUM_WORKERS = 48

# Model
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(mutaSCAN_metadata.thing_classes)

# Solver
cfg.SOLVER.BASE_LR = 0.0003
cfg.SOLVER.MAX_ITER = 6000
cfg.SOLVER.STEPS = (2000, 4000)
cfg.SOLVER.GAMMA = 0.3
cfg.SOLVER.WARMUP_FACTOR = 0.001
cfg.SOLVER.WARMUP_ITERS = 200
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.CHECKPOINT_PERIOD = 100

# Test
cfg.TEST.DETECTIONS_PER_IMAGE = 100
cfg.TEST.EVAL_PERIOD = 100
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75

# DATASETS
cfg.DATASETS.TRAIN = ('mutaSCAN_train',)
cfg.DATASETS.TEST = ('mutaSCAN_valid',)

# Output Model
cfg.OUTPUT_DIR = str(utility.generate_timedate_cache_file(OUTPUT_DIR, suffix=f'{n_train_samples}_samples'))

In [ ]:
MetadataCatalog.get(cfg.DATASETS.TRAIN[0])

In [ ]:
from detectron2.data.catalog import Metadata

In [ ]:
Metadata(name='', thing_classes=['0', '1', '2'])

Visualize Augmentations

In [ ]:
transform_list = [
    T.RandomBrightness(0.35, 1.6),
    T.RandomContrast(0.25, 1.6),
    T.RandomSaturation(0.25, 1.4),
    T.RandomRotation(angle=[0, 180]),
    T.RandomLighting(0.7),
    T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
]

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=CustomMapper(cfg, transform_list))

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
            utility.create_directory(cfg.OUTPUT_DIR)
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                CustomMapper(cfg, [])
            )
        ))
        return hooks

mapper = CustomMapper(cfg, transform_list)

In [ ]:
config_name = 'CustomTrainer_config.yaml'
trainer = CustomTrainer(cfg)

Start to Train

In [ ]:
utility.create_directory(cfg.OUTPUT_DIR)
with open(Path(cfg.OUTPUT_DIR) / Path(config_name), 'w') as f:
    f.write(cfg.dump())
trainer.resume_or_load(resume=True)
trainer.train()

Use model for inference

In [ ]:
cfg.MODEL.WEIGHTS = str(OUTPUT_DIR / Path('20230316_195019_153387_5_samples') / Path('model_final.pth'))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg, )

In [ ]:
im = cv2.imread(glob('caches/test_samples/*_sample2_9*.png')[0])

outputs = predictor(im)
print(outputs["instances"].pred_classes)

v = Visualizer(im, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1, instance_mode=ColorMode.IMAGE)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image(), bgr2rgb=True)

In [ ]:
minute = 10
for i in range(6):
    im = cv2.imread(glob(f'caches/test_samples/*_sample{i + 1}_{minute - 1}*.png')[0])
    outputs = predictor(im)
    print(outputs["instances"].pred_classes)
    v = Visualizer(im, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1, instance_mode=ColorMode.IMAGE)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image(), bgr2rgb=True)

In [ ]:
minute = 30
for i in range(6):
    im = cv2.imread(glob(f'caches/test_samples/*_sample{i + 1}_{minute - 1}*.png')[0])
    outputs = predictor(im)
    print(outputs["instances"].pred_classes)
    v = Visualizer(im, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1, instance_mode=ColorMode.IMAGE)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image(), bgr2rgb=True)

In [ ]:
experiment_folder = './output/model_iter4000_lr0005_wf1_date2020_03_20__05_16_45'

def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

experiment_metrics = load_json_arr(experiment_folder + '/metrics.json')

plt.plot(
    [x['iteration'] for x in experiment_metrics],
    [x['total_loss'] for x in experiment_metrics])
plt.plot(
    [x['iteration'] for x in experiment_metrics if 'validation_loss' in x],
    [x['validation_loss'] for x in experiment_metrics if 'validation_loss' in x])
plt.legend(['total_loss', 'validation_loss'], loc='upper left')
plt.show()